__--outline--__

In [55]:
# Title
# from The National Renewable Energy Laboratory (NREL) Rooftop Energy Potential of Low Income Communities in America (REPLICA)

# Abstract
# ...analysis of homes qualified for rooftop solar deployment; supplementary spatial data analysi in Tableau

# Scope (notebook)
# CSV->SQLite3 query->Pandas subset->CSV(s)

# Setup
# import

# Obtain
# CSV; columns 

# Scrub 
# create db; verify table(s)

# Explore
# xxxsolar suitablexxx # disregard for this data source
# owner occupied
# subsidies available

# Model
#

# References
#

## Setup

In [2]:
import os
import numpy as np
import pandas as pd
import sqlite3

import matplotlib.pyplot as plt
%matplotlib inline

## Obtain

In [3]:
# listing files in the current directory
os.listdir()

['.git', '.gitignore', '.ipynb_checkpoints', 'data', 'README.md', 'sol.ipynb']

In [4]:
# previewing data directory
os.listdir('data/')

['.ipynb_checkpoints',
 'archive',
 'rep.sqlite',
 'SEEDSII REPLICA Documentation.pdf',
 'seeds_ii_replica.csv']

Our dataset originates from:\ Mooney, Meghan; Sigrin, Ben (2018): Rooftop Energy Potential of Low Income Communities in America REPLICA. National Renewable Energy Laboratory. https://data.nrel.gov/submissions/81

The CSV flat file comprises a variety of sources. We will instantiate the data as a Pandas dataframe and view the first rows.

In [5]:
# 'https://data.nrel.gov/system/files/81/seeds_ii_replica%20%281%29.csv'
data_df = pd.read_csv('data/seeds_ii_replica.csv', nrows=10)

# viewing first rows
data_df.head()

,geoid,gisjoin,state_fips,state_name,state_abbr,county_name,county_fips,tract_fips,area_km2,centroid_x,...,climate_zone_description,moisture_regime,locale,total_units,active_subsidies,avg_months_tenancy,fmr_2br,occ_rate,pct_eli_hh,lihtc_qualified
0,6001400100,G0600010400100,6,California,CA,Alameda County,1,400100,6.894336,-122.231947,...,Warm,C,City - Large,NaN,NaN,NaN,NaN,NaN,NaN,False
1,6001408100,G0600010408100,6,California,CA,Alameda County,1,408100,8.935563,-122.157212,...,Warm,C,City - Large,NaN,NaN,NaN,NaN,NaN,NaN,False
2,6001409000,G0600010409000,6,California,CA,Alameda County,1,409000,16.927907,-122.212192,...,Warm,C,City - Large,NaN,NaN,NaN,NaN,NaN,NaN,True
3,6001409900,G0600010409900,6,California,CA,Alameda County,1,409900,4.332550,-122.135826,...,Warm,C,City - Large,NaN,NaN,NaN,NaN,NaN,NaN,False
4,6001422000,G0600010422000,6,California,CA,Alameda County,1,422000,4.727481,-122.302847,...,Warm,C,City - Midsize,NaN,NaN,NaN,NaN,NaN,NaN,True


In [6]:
# viewing all attribute headers
for col in data_df.columns:
    print(col)

geoid
gisjoin
state_fips
state_name
state_abbr
county_name
county_fips
tract_fips
area_km2
centroid_x
very_low_mf_own_hh
very_low_mf_rent_hh
very_low_sf_own_hh
very_low_sf_rent_hh
low_mf_own_hh
low_mf_rent_hh
low_sf_own_hh
low_sf_rent_hh
mod_mf_own_hh
mod_mf_rent_hh
mod_sf_own_hh
mod_sf_rent_hh
mid_mf_own_hh
mid_mf_rent_hh
mid_sf_own_hh
mid_sf_rent_hh
high_mf_own_hh
high_mf_rent_hh
high_sf_own_hh
high_sf_rent_hh
very_low_mf_own_bldg_cnt
very_low_mf_rent_bldg_cnt
very_low_sf_own_bldg_cnt
very_low_sf_rent_bldg_cnt
low_mf_own_bldg_cnt
low_mf_rent_bldg_cnt
low_sf_own_bldg_cnt
low_sf_rent_bldg_cnt
mod_mf_own_bldg_cnt
mod_mf_rent_bldg_cnt
mod_sf_own_bldg_cnt
mod_sf_rent_bldg_cnt
mid_mf_own_bldg_cnt
mid_mf_rent_bldg_cnt
mid_sf_own_bldg_cnt
mid_sf_rent_bldg_cnt
high_mf_own_bldg_cnt
high_mf_rent_bldg_cnt
high_sf_own_bldg_cnt
high_sf_rent_bldg_cnt
very_low_mf_own_devp_cnt
very_low_mf_rent_devp_cnt
very_low_sf_own_devp_cnt
very_low_sf_rent_devp_cnt
low_mf_own_devp_cnt
low_mf_rent_devp_cnt
low_sf_

## Scrub

### Create SQLite3 Database

In [7]:
# create a new database file "conn":
conn = sqlite3.connect("data/rep.sqlite")

In [8]:
# load the CSV in chunks:
for row in pd.read_csv(
    "data/seeds_ii_replica.csv", chunksize=1000):
    # append each row to a new table "seed" in the database:
    row.to_sql("seed", conn, if_exists="append")

    # index on the 'geoid' column:
conn.execute("CREATE INDEX geoid ON seed(geoid)") 
conn.close()

### connect to sqlite database

In [9]:
# creating a connection to the database
conn = sqlite3.connect("data/rep.sqlite")

# instantiating a cursor
cur = conn.cursor()

### listing the sqlite table names

In [10]:
# getting names of all tables in the sql database
sql_tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")

# assign table names to a variable as a list
table_list = list(map(lambda x: x[0], sql_tables.fetchall()))

# viewing/verifying table names
table_list

['seed']

Correctly, `seed` is shown as the only table in the database.

## Explore

We can review our data description to select an area of investigation

### Custom Query

Let's find all of `data_df`'s column names for owner occupied properties that are listed as suitable buildings (those including the string "own_bldg_cnt").

In [31]:
# column names for owner-occupied, solar-suitable buildings
owner_suitable = [col for col in data_df.columns if "own_bldg_cnt" in col]
owner_suitable

['very_low_mf_own_bldg_cnt',
 'very_low_sf_own_bldg_cnt',
 'low_mf_own_bldg_cnt',
 'low_sf_own_bldg_cnt',
 'mod_mf_own_bldg_cnt',
 'mod_sf_own_bldg_cnt',
 'mid_mf_own_bldg_cnt',
 'mid_sf_own_bldg_cnt',
 'high_mf_own_bldg_cnt',
 'high_sf_own_bldg_cnt']

The columns listed in `owner_suitable` report the total annual solar power generation potential (MWh) for solar-suitable buildings.

Let's query these data for tracts with active subsidies.

In [28]:
# creating a connection to the database
conn = sqlite3.connect("data/rep.sqlite")

# instantiating a cursor
cur = conn.cursor()

In [29]:
# querying attributes
cur.execute("""SELECT geoid, gisjoin, 
                state_abbr, county_name, pop_total, high_sf_own_hh,
                active_subsidies, avg_monthly_bill_dlrs, dlrs_kwh,
                very_low_mf_own_bldg_cnt, very_low_sf_own_bldg_cnt, 
                low_mf_own_bldg_cnt, low_sf_own_bldg_cnt, 
                mod_mf_own_bldg_cnt, mod_sf_own_bldg_cnt, 
                mid_mf_own_bldg_cnt, mid_sf_own_bldg_cnt, 
                high_mf_own_bldg_cnt, high_sf_own_bldg_cnt
               FROM seed
               WHERE active_subsidies > 0
            """);

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]

# exiting database connection    
conn.close()

# previewing the dataframe
df

,geoid,gisjoin,state_abbr,county_name,pop_total,very_low_mf_own_hh,active_subsidies,avg_monthly_bill_dlrs,dlrs_kwh,very_low_mf_own_bldg_cnt,very_low_sf_own_bldg_cnt,low_mf_own_bldg_cnt,low_sf_own_bldg_cnt,mod_mf_own_bldg_cnt,mod_sf_own_bldg_cnt,mid_mf_own_bldg_cnt,mid_sf_own_bldg_cnt,high_mf_own_bldg_cnt,high_sf_own_bldg_cnt
0,41017000500,G4100170000500,OR,Deschutes County,5847,0.0,1.0,118.688180,0.088055,0.2681,85.9520,1.3405,67.1244,1.8767,130.1559,1.6086,172.7225,6.3077,698.2575
1,41017000800,G4100170000800,OR,Deschutes County,6909,2.0,4.0,101.412973,0.110601,0.3102,165.8723,0.2979,104.7614,2.6930,239.5933,2.9068,278.3938,6.6598,633.4187
2,41017000900,G4100170000900,OR,Deschutes County,7495,3.0,13.0,101.412973,0.110601,2.7129,341.7480,2.1251,205.4382,3.9131,267.7513,2.5207,294.0396,4.5255,355.3789
3,53033025006,G5300330025006,WA,King County,6008,3.0,2.0,96.376683,0.111160,0.8993,21.1499,1.3617,28.1999,1.3257,35.2499,4.0874,100.2664,25.9989,785.6813
4,53033027701,G5300330027701,WA,King County,5565,13.0,7.0,96.376683,0.111160,3.0984,69.5288,5.6797,100.4304,4.0178,117.4264,4.2618,149.8731,15.2504,444.2116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25690,45019002802,G4500190002802,SC,Charleston County,2726,2.0,1.0,163.098857,0.145642,0.6422,21.5491,0.6421,14.9186,1.1227,53.8727,3.6657,96.1420,9.6551,307.4886
25691,27139080302,G2701390080302,MN,Scott County,15090,0.0,3.0,83.731445,0.131830,5.3075,86.5339,10.6150,158.2085,13.9900,270.9649,38.4172,578.6411,114.7385,1812.8425
25692,34009021300,G3400090021300,NJ,Cape May County,3957,11.0,1.0,123.070925,0.173899,5.3800,90.2319,8.2768,145.4222,12.5307,254.9269,11.8824,176.0835,17.5528,260.1831
25693,34023008900,G3400230008900,NJ,Middlesex County,4004,8.0,1.0,97.396944,0.164645,3.2189,6.0137,7.5152,12.3071,5.2842,8.1404,8.1484,16.9270,14.0107,29.7838


In [14]:
# # exiting database connection    
conn.close()

In [15]:
# viewing info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25695 entries, 0 to 25694
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   geoid                     25695 non-null  int64  
 1   gisjoin                   25695 non-null  object 
 2   state_abbr                25695 non-null  object 
 3   county_name               25695 non-null  object 
 4   pop_total                 25695 non-null  int64  
 5   active_subsidies          25695 non-null  float64
 6   avg_monthly_bill_dlrs     25695 non-null  float64
 7   dlrs_kwh                  25695 non-null  float64
 8   very_low_mf_own_bldg_cnt  25695 non-null  float64
 9   very_low_sf_own_bldg_cnt  25695 non-null  float64
 10  low_mf_own_bldg_cnt       25695 non-null  float64
 11  low_sf_own_bldg_cnt       25695 non-null  float64
 12  mod_mf_own_bldg_cnt       25695 non-null  float64
 13  mod_sf_own_bldg_cnt       25695 non-null  float64
 14  mid_mf

In [16]:
# summing unique entries for each column
df.nunique()

geoid                       25695
gisjoin                     25695
state_abbr                     44
county_name                  1629
pop_total                    7568
active_subsidies               39
avg_monthly_bill_dlrs        1023
dlrs_kwh                     1023
very_low_mf_own_bldg_cnt    17470
very_low_sf_own_bldg_cnt    25308
low_mf_own_bldg_cnt         15957
low_sf_own_bldg_cnt         25173
mod_mf_own_bldg_cnt         19311
mod_sf_own_bldg_cnt         25346
mid_mf_own_bldg_cnt         20096
mid_sf_own_bldg_cnt         25359
high_mf_own_bldg_cnt        22825
high_sf_own_bldg_cnt        25443
dtype: int64

---

We have ten owner-occupied building categories (query result columns 7-16). Let's engineer a calculated column to sum the total number of owner occupied buildings in each tract.

In [17]:
# listing the columns to sum (col 7-through-last)
cols_to_sum = df.columns[ 7:]
cols_to_sum

Index(['dlrs_kwh', 'very_low_mf_own_bldg_cnt', 'very_low_sf_own_bldg_cnt',
       'low_mf_own_bldg_cnt', 'low_sf_own_bldg_cnt', 'mod_mf_own_bldg_cnt',
       'mod_sf_own_bldg_cnt', 'mid_mf_own_bldg_cnt', 'mid_sf_own_bldg_cnt',
       'high_mf_own_bldg_cnt', 'high_sf_own_bldg_cnt'],
      dtype='object')

In [18]:
# adding a column to the dataframe ("all_own")
# summing `cols_to_sum` for each row in the dataframe
df['all_own'] = df[cols_to_sum].sum(axis=1)

In [19]:
# reviewing summary statistics
df.describe()

,geoid,pop_total,active_subsidies,avg_monthly_bill_dlrs,dlrs_kwh,very_low_mf_own_bldg_cnt,very_low_sf_own_bldg_cnt,low_mf_own_bldg_cnt,low_sf_own_bldg_cnt,mod_mf_own_bldg_cnt,mod_sf_own_bldg_cnt,mid_mf_own_bldg_cnt,mid_sf_own_bldg_cnt,high_mf_own_bldg_cnt,high_sf_own_bldg_cnt,all_own
count,2.569500e+04,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000
mean,3.301156e+10,4483.792683,3.307647,111.248988,0.128887,2.252963,91.658702,1.814383,60.059984,2.930271,118.653164,3.884647,144.306591,8.936776,332.245147,766.871513
std,1.276324e+10,2091.767106,3.092658,22.411580,0.034146,3.286216,60.653672,2.573076,39.510449,3.680485,72.110110,5.132888,92.991432,13.069552,272.144669,454.802081
min,1.000104e+10,0.000000,1.000000,46.845787,0.029553,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.101024
25%,2.206250e+10,3031.000000,1.000000,97.396944,0.107758,0.554200,48.177950,0.287400,32.391250,0.717850,67.934250,0.834150,78.252900,1.654650,133.461350,454.930060
50%,3.403903e+10,4206.000000,2.000000,111.794393,0.118972,1.323000,81.199500,1.002100,53.732200,1.850800,111.936400,2.179700,134.853400,4.339100,285.136500,726.828840
75%,4.210101e+10,5593.000000,4.000000,127.104540,0.138892,2.804850,123.687550,2.357900,80.739650,3.880500,159.741800,4.998850,196.115200,10.673550,462.297350,1020.557872
max,5.604595e+10,35527.000000,50.000000,262.413841,0.340253,73.180300,750.564300,56.182500,457.833300,100.469100,791.294500,99.887400,1363.645500,218.978500,3964.257500,6250.145065


---

### Subsidies

In [20]:
# finding tracts with the greatest number of subsidies
df[df['active_subsidies']==df['active_subsidies'].max()]

,geoid,gisjoin,state_abbr,county_name,pop_total,active_subsidies,avg_monthly_bill_dlrs,dlrs_kwh,very_low_mf_own_bldg_cnt,very_low_sf_own_bldg_cnt,low_mf_own_bldg_cnt,low_sf_own_bldg_cnt,mod_mf_own_bldg_cnt,mod_sf_own_bldg_cnt,mid_mf_own_bldg_cnt,mid_sf_own_bldg_cnt,high_mf_own_bldg_cnt,high_sf_own_bldg_cnt,all_own
20232,36061023000,G3600610023000,NY,New York County,9259,50.0,98.685298,0.249082,1.2661,3.0588,0.7864,1.1471,1.2572,2.2941,1.2882,1.1471,1.7296,1.5294,15.753082


In [21]:
# subset columns from tracts with the lowest number of subsidies
df.loc[df['active_subsidies']==
       df['active_subsidies'].min(), 
       ['geoid', 'state_abbr', 'county_name', 'very_low_mf_own_bldg_cnt']]

,geoid,state_abbr,county_name,very_low_mf_own_bldg_cnt
0,41017000500,OR,Deschutes County,0.2681
5,28029950500,MS,Copiah County,0.0000
6,28033070323,MS,DeSoto County,0.9087
7,28033071110,MS,DeSoto County,0.8272
13,28043950200,MS,Grenada County,0.0000
...,...,...,...,...
25679,12087971200,FL,Monroe County,2.9583
25688,12086003915,FL,Miami-Dade County,5.4488
25690,45019002802,SC,Charleston County,0.6422
25692,34009021300,NJ,Cape May County,5.3800


### Solar suitable buildings per capita

Before we can calculate the number of qualified buildings per capita, we need to better understand a few of our columns. 

The data documentation indicates that the `...bldg_cnt` columns represent building counts. However, as we inspect the data we observe that the values are floats.

What does it mean that Oregon's Deschutes County has a 'Very Low Income (0-30% AMI) - Single-Family -
Owner-Occupied - Household Count' of 0.2681?

(The data attributes in question are estimates.)

---

===== code for dropping the table (e.g., to avoid duplicating records if re-running the notebook) =====

In [22]:
# # creating a connection to the database
# conn = sqlite3.connect("data/rep.sqlite")

# # instantiating a cursor
# cur = conn.cursor()

# # dropping the table
# cur.execute("""DROP TABLE seed
#             """);

In [28]:
# creating a connection to the database
conn = sqlite3.connect("data/rep.sqlite")

# instantiating a cursor
cur = conn.cursor()

=======================================
=

## Capacity and Generation

Singe-family, owner-occupied households in states with active subsidies.

In [32]:
# listing column names for owner-occupied households
own_cols = [col for col in data_df.columns if "own" in col]
own_cols

['very_low_mf_own_hh',
 'very_low_sf_own_hh',
 'low_mf_own_hh',
 'low_sf_own_hh',
 'mod_mf_own_hh',
 'mod_sf_own_hh',
 'mid_mf_own_hh',
 'mid_sf_own_hh',
 'high_mf_own_hh',
 'high_sf_own_hh',
 'very_low_mf_own_bldg_cnt',
 'very_low_sf_own_bldg_cnt',
 'low_mf_own_bldg_cnt',
 'low_sf_own_bldg_cnt',
 'mod_mf_own_bldg_cnt',
 'mod_sf_own_bldg_cnt',
 'mid_mf_own_bldg_cnt',
 'mid_sf_own_bldg_cnt',
 'high_mf_own_bldg_cnt',
 'high_sf_own_bldg_cnt',
 'very_low_mf_own_devp_cnt',
 'very_low_sf_own_devp_cnt',
 'low_mf_own_devp_cnt',
 'low_sf_own_devp_cnt',
 'mod_mf_own_devp_cnt',
 'mod_sf_own_devp_cnt',
 'mid_mf_own_devp_cnt',
 'mid_sf_own_devp_cnt',
 'high_mf_own_devp_cnt',
 'high_sf_own_devp_cnt',
 'very_low_mf_own_devp_m2',
 'very_low_sf_own_devp_m2',
 'low_mf_own_devp_m2',
 'low_sf_own_devp_m2',
 'mod_mf_own_devp_m2',
 'mod_sf_own_devp_m2',
 'mid_mf_own_devp_m2',
 'mid_sf_own_devp_m2',
 'high_mf_own_devp_m2',
 'high_sf_own_devp_m2',
 'very_low_mf_own_mw',
 'very_low_sf_own_mw',
 'low_mf_own_mw'

We will look at high, mid, low, and very-low income households and compare their potential as well as their annual power output. 

The `mw` columns refer to maximum solar-power capacity at any point in time. The `mwh` column indicates the amount of solar electricity generated from photovoltaic rooftop panels in a year's time.

### Query the Desired Columns 

In [48]:
# creating a connection to the database
conn = sqlite3.connect("data/rep.sqlite")

# instantiating a cursor
cur = conn.cursor()

In [49]:
# querying attributes
cur.execute("""SELECT geoid, gisjoin, state_abbr, 
                county_name, pop_total, hu_own, 
                high_sf_own_hh, high_sf_own_mw, 
                high_sf_own_mwh, 
                mid_sf_own_hh, mid_sf_own_mw, 
                mid_sf_own_mwh, 
                low_sf_own_hh, low_sf_own_mw, 
                low_sf_own_mwh, 
                very_low_sf_own_hh, very_low_sf_own_mw, 
                very_low_sf_own_mwh, 
                active_subsidies
               FROM seed
               WHERE active_subsidies > 0
            """);

own_df = pd.DataFrame(cur.fetchall())
own_df.columns = [x[0] for x in cur.description]

# exiting database connection    
conn.close()

# previewing the dataframe
own_df

,geoid,gisjoin,state_abbr,county_name,pop_total,hu_own,high_sf_own_hh,high_sf_own_mw,high_sf_own_mwh,mid_sf_own_hh,mid_sf_own_mw,mid_sf_own_mwh,low_sf_own_hh,low_sf_own_mw,low_sf_own_mwh,very_low_sf_own_hh,very_low_sf_own_mw,very_low_sf_own_mwh,active_subsidies
0,41017000500,G4100170000500,OR,Deschutes County,5847,1138,870.0,6.154685,7542.147835,216.0,1.522437,1865.642665,85.0,0.591658,725.036486,106.0,0.757611,928.400378,1.0
1,41017000800,G4100170000800,OR,Deschutes County,6909,1063,665.0,5.537467,7012.639190,294.0,2.433772,3082.124728,109.0,0.915844,1159.823940,171.0,1.450087,1836.387904,4.0
2,41017000900,G4100170000900,OR,Deschutes County,7495,605,374.0,3.059528,3900.588252,309.0,2.531445,3227.336033,214.0,1.768659,2254.860606,358.0,2.942177,3750.976648,13.0
3,53033025006,G5300330025006,WA,King County,6008,1258,1062.0,6.974679,6150.765841,140.0,0.890089,784.943198,40.0,0.250337,220.765275,30.0,0.187753,165.573956,2.0
4,53033027701,G5300330027701,WA,King County,5565,1191,608.0,3.822506,3614.035306,205.0,1.289680,1219.344086,145.0,0.864219,817.086243,95.0,0.598305,565.675092,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25690,45019002802,G4500190002802,SC,Charleston County,2726,563,399.0,2.608841,3278.625685,126.0,0.815702,1025.122856,20.0,0.126574,159.070788,27.0,0.182830,229.768916,1.0
25691,27139080302,G2701390080302,MN,Scott County,15090,3460,2461.0,15.600179,17540.772929,794.0,4.979421,5598.838804,218.0,1.361443,1530.800338,118.0,0.744657,837.288583,3.0
25692,34009021300,G3400090021300,NJ,Cape May County,3957,816,332.0,2.222304,2775.957039,224.0,1.503984,1878.677996,182.0,1.242096,1551.545012,115.0,0.770698,962.705640,1.0
25693,34023008900,G3400230008900,NJ,Middlesex County,4004,410,372.0,0.220280,281.252156,220.0,0.125191,159.843766,162.0,0.091022,116.216891,76.0,0.044477,56.787801,1.0


We will save this subset as a csv.

In [51]:
# saving "own_df" as a CSV file
own_df.to_csv('data/own.csv', chunksize=1000)

--> WIP <--
=

    TO DOS:
>
>    * Add region
>    * Static plotting visual EDA
>    * Interactive plotting
>    * Tableau visualization/dashboard